# Unteruschung der Extraktionsqualität

In [1]:
import os
import pandas as pd
from ddbapi import zp_pages

places = ["Köln"]

outdir = "data"
if not os.path.isdir(outdir):
    os.mkdir(outdir)
#Save all the datasets individually - so we have a checkpoint for the org. data
df_list = []
for place in places:
    df = zp_pages(
        publication_date='[1850-01-01T12:00:00Z TO 1980-12-31T12:00:00Z]', 
        place_of_distribution=place, 
        plainpagefulltext=["Heiratsgesuch"]
        )
    
    df_list.append(df)

df_final = pd.concat(df_list)
df_final.to_pickle("ddbapi_data_koeln.pkl")

https://api.deutsche-digitale-bibliothek.de/search/index/newspaper-issues/select?rows=1000&sort=id+ASC&q=type%3Apage+AND+publication_date%3A%22%5B1850-01-01T12%3A00%3A00Z%5C+TO%5C+1980-12-31T12%3A00%3A00Z%5D%22+AND+place_of_distribution%3A%22K%C3%B6ln%22+AND+%28plainpagefulltext%3AHeiratsgesuch%29&cursorMark=%2A
Getting 1000 of 2696
Getting 2000 of 2696
Getting 2696 of 2696
Got 2696 items.


In [4]:
import pandas as pd

df_final = pd.read_pickle("ddbapi_data_koeln.pkl")

In [10]:
from scripts.information_extractor_lib import InformationExtractor

information_extractor = InformationExtractor(df=df_final, page_id_colname="page_id", text_colname="plainpagefulltext", output_filename="out_df.csv")
information_extractor.save_one_result_per_row = False
ergebnis_df = information_extractor.extract_data_loop(max_n = 10)
print(ergebnis_df)

                                             page_id  \
0  22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...   
1  23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...   
2  23JRMGNQTVLEGWUXLZ6D34KX4C4GCJNH-ALTO7691058_D...   
3  245BHDGA6W5OOYPCWEGLHABY6LM7YNXP-ALTO7758526_D...   
4  24KXQUATL73II54JILODDITR7CKGE7L4-ALTO2320495_D...   
5  2537G7CWASBPS3RB4D6HOOJCTI3X6P24-ALTO372289_DD...   
6  254IA3LVBUBODIO6NDVHDOJOA654B5KT-ALTO10118878_...   
7  257JWTXYMDZQIF4DFV67FVD6FCP7MNN3-ALTO10158655_...   
8  25EFYTTDJ6ZZXEVF33JRQTD6I3UXILDC-ALTO8101159_D...   
9  25FPH6FZUIBNZJYXMXXDMC7CKZH6Q7E6-ALTO8702702_D...   

                                                text              time  
0  [Suche für meine Tochter , 18 Jahre , Stelle a...  2025-06-27_11_22  
1  [Tehal nach Brükcnes . Gewissenhafter deutsche...  2025-06-27_11_22  
2  [Suche f . meine Nichte ohne d . Wissen S tein...  2025-06-27_11_22  
3  [Energischer Reisender . , 27 . , sucht Engage...  2025-06-27_11_22  
4  [Das kleine „He

In [9]:
information_extractor.save_one_result_per_row 

True

# Unnesting the Data - because per row multiple results are saved

In [6]:
def unnest_data(ergebnis_df:pd.DataFrame) -> pd.DataFrame:
    erg_list = []
    for i in range(len(ergebnis_df)):
        temp_list = []
        row = ergebnis_df.iloc[i,:]
        for t in row["text"]:
            temp_list.append(t)
        n = len(temp_list)
        df_temp = pd.DataFrame(data={"page_id":[row["page_id"]]*n, "text":temp_list,"time":[row["time"]]*n})
        erg_list.append(df_temp)
    df_unnested = pd.concat(erg_list)
    return df_unnested.reset_index().filter(["page_id", "text", "time"])



In [11]:
full_data = unnest_data(ergebnis_df)

In [12]:
full_data

,page_id,text,time
0,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Suche für meine Tochter , 18 Jahre , Stelle al...",2025-06-27_11_22
1,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...,2025-06-27_11_22
2,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Junger Mann . 2 J hre , kath . , sucht aufrich...",2025-06-27_11_22
3,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Zwei junge Herren , 19 # . 21 Jahre , suchen d...",2025-06-27_11_22
4,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,Tehal nach Brükcnes . Gewissenhafter deutscher...,2025-06-27_11_22
5,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"Sr Rittergutsbesitzers - Sohn 30 J . alt , Erb...",2025-06-27_11_22
6,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"Heiratsgesuch . Ingenieur , 31 Jahre alt , in ...",2025-06-27_11_22
7,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,Gener . - Verteckung dir Gat Aloue Gomecher Gl...,2025-06-27_11_22
8,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"IVritwe . kath . , m . 2 . , wünscht d . Bekan...",2025-06-27_11_22
9,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"teiraf 27jährig . Herr , ev . , aus erster rhe...",2025-06-27_11_22


In [13]:
unique_page_ids = full_data[full_data.filter(["page_id"]).duplicated() == False]["page_id"].to_list()

## Berechnung der Accuracy der Informationsextraction 

Es wird für jeden Full-Text für Berechnet ob die entnommenen Kontaktanzeigen 1:1 genau so im Volltext enthalten sind wie sie von der GeminiAPI entnommen werden. Es wird ein über die Anzahl der entnommenen Anzeigen gewichteter Mittelwert der Accuracy berechnet.
  

In [14]:
n = len(full_data)
total_extracted = 0
weighted_perfect_extraction_ratio = 0
for i in range(len(unique_page_ids)):

    curr_full_text = df_final[df_final.page_id == unique_page_ids[i]].plainpagefulltext.tolist()[0]
    exctracted = full_data[full_data.page_id == unique_page_ids[i]].text.tolist()
    perfect_extraction_ratio = sum([e in curr_full_text for e in exctracted])/len(exctracted)
    total_extracted += sum([e in curr_full_text for e in exctracted])
    weighted_perfect_extraction_ratio += perfect_extraction_ratio*(len(exctracted)/n)



    #print(f"Extraction Ratio {perfect_extraction_ratio}, Len of extracted {len(exctracted)} ")

print(f"Total perfect_extraction_ratio weighted {weighted_perfect_extraction_ratio}")
print(f"Total extraction rate: {total_extracted/n}")






Total perfect_extraction_ratio weighted 0.8043478260869564
Total extraction rate: 0.8043478260869565


# Untersuchen ob es kleinere Fehler gibt, die nicht den Sinn/Struktur verändern.

Prüfen ob der String mit einer annähernden Gleichheit im Originaltext vorhanden ist.

In [15]:
df_red = df_final.filter(["page_id", "plainpagefulltext"])

dat_joined = pd.merge(full_data, df_red, on="page_id", how="left")
dat_joined

,page_id,text,time,plainpagefulltext
0,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Suche für meine Tochter , 18 Jahre , Stelle al...",2025-06-27_11_22,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
1,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...,2025-06-27_11_22,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
2,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Junger Mann . 2 J hre , kath . , sucht aufrich...",2025-06-27_11_22,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
3,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Zwei junge Herren , 19 # . 21 Jahre , suchen d...",2025-06-27_11_22,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
4,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,Tehal nach Brükcnes . Gewissenhafter deutscher...,2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
5,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"Sr Rittergutsbesitzers - Sohn 30 J . alt , Erb...",2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
6,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"Heiratsgesuch . Ingenieur , 31 Jahre alt , in ...",2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
7,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,Gener . - Verteckung dir Gat Aloue Gomecher Gl...,2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
8,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"IVritwe . kath . , m . 2 . , wünscht d . Bekan...",2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
9,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"teiraf 27jährig . Herr , ev . , aus erster rhe...",2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."


In [16]:
not_correct_extracted_df = dat_joined[dat_joined.apply(lambda row: row["text"] in row["plainpagefulltext"], axis = 1) == False]
not_correct_extracted_df

,page_id,text,time,plainpagefulltext
3,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Zwei junge Herren , 19 # . 21 Jahre , suchen d...",2025-06-27_11_22,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
9,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"teiraf 27jährig . Herr , ev . , aus erster rhe...",2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
22,245BHDGA6W5OOYPCWEGLHABY6LM7YNXP-ALTO7758526_D...,"Heirat . Dr . , Apotheker u . Chemik nf . 30er...",2025-06-27_11_22,"Soennecken - Federn Das beste , was die Schrei..."
23,245BHDGA6W5OOYPCWEGLHABY6LM7YNXP-ALTO7758526_D...,"Eine junge , sehr hübsche Witwe reidenk . , gä...",2025-06-27_11_22,"Soennecken - Federn Das beste , was die Schrei..."
26,24KXQUATL73II54JILODDITR7CKGE7L4-ALTO2320495_D...,Das kleine „Heiratsgesuch“ in der illustrierte...,2025-06-27_11_22,Ein Bauer kehrt heim Erzählung von Wilhelm Len...
30,254IA3LVBUBODIO6NDVHDOJOA654B5KT-ALTO10118878_...,"Netles Heiratsgesuch . Junggeselle vom Lande ,...",2025-06-27_11_22,W V a . Früchtsettauf in Ranzel . Die Geschwis...
32,257JWTXYMDZQIF4DFV67FVD6FCP7MNN3-ALTO10158655_...,Heiratsgesuch . ver g . Kfr Sbnss Kreis geoun ...,2025-06-27_11_22,22 . SEPTEMBER 1929 DIE LITERATUR AUS VERSCHIE...
41,25FPH6FZUIBNZJYXMXXDMC7CKZH6Q7E6-ALTO8702702_D...,"Heirat .\nFür zwei junge Damen chlerung , Töch...",2025-06-27_11_22,"fufschhe Allerersten Ranges , Inmitten eines e..."
43,25FPH6FZUIBNZJYXMXXDMC7CKZH6Q7E6-ALTO8702702_D...,"Di1 uche für einen Verwandt Architekt , Mitinh...",2025-06-27_11_22,"fufschhe Allerersten Ranges , Inmitten eines e..."


In [17]:
dat_joined[dat_joined.apply(lambda row: row["text"] in row["plainpagefulltext"], axis=1)]

,page_id,text,time,plainpagefulltext
0,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Suche für meine Tochter , 18 Jahre , Stelle al...",2025-06-27_11_22,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
1,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...,2025-06-27_11_22,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
2,22KVFOV36KTZS67POXRNU5DJKD4XYSPI-ALTO1934911_D...,"Junger Mann . 2 J hre , kath . , sucht aufrich...",2025-06-27_11_22,1 — 92 . R Geicch * 1 RRAR ädchen Schuhmacherg...
4,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,Tehal nach Brükcnes . Gewissenhafter deutscher...,2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
5,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"Sr Rittergutsbesitzers - Sohn 30 J . alt , Erb...",2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
6,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"Heiratsgesuch . Ingenieur , 31 Jahre alt , in ...",2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
7,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,Gener . - Verteckung dir Gat Aloue Gomecher Gl...,2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
8,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"IVritwe . kath . , m . 2 . , wünscht d . Bekan...",2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
10,23GSP3TO42E3USTLALB2X5EHUMBSEOWM-ALTO8960309_D...,"Reelle Heiratsgesuch ! Suche f . Verwandt . , ...",2025-06-27_11_22,"Mittwoch , 4 . Juni in den Ver . Staten aufero..."
11,23JRMGNQTVLEGWUXLZ6D34KX4C4GCJNH-ALTO7691058_D...,Suche f . meine Nichte ohne d . Wissen S teine...,2025-06-27_11_22,Orzug altbewährten MAGGI WURZE 1 . Feinstes Ar...


Laufen lassen des Strings über den Ausgangstext und extrahieren der Sequencen die eine Stringsimiliarity (Levenstein Distance) von größer 0.98 haben um zu schauen, ob wir approximativ die gleichen Texte haben.

In [18]:
dat_joined.to_pickle("dat_joined.pkl")

In [21]:
from rapidfuzz import fuzz

def find_similiar_sequence(dst_doc, source_doc, threshold:float):

    n = len(dst_doc)
    n_src = len(source_doc)
    i = 0

    best_match_list = []
    best_ratio_list = []
    while n < n_src:
        stringsim = fuzz.ratio(dst_doc, source_doc[i:n])/100
        if stringsim > threshold: 
            best_match_list.append(source_doc[i:n])
            best_ratio_list.append(stringsim)
        i += 1
        n += 1

    matches_df = pd.DataFrame(data={"ratio":best_ratio_list, "match":best_match_list, "extracted":dst_doc})
    if len(matches_df) > 0:
        return True, matches_df
    else: 
        return False
    

investigation_idx = 1
dst_doc, source_doc = not_correct_extracted_df.reset_index().text[investigation_idx], not_correct_extracted_df.reset_index().plainpagefulltext[investigation_idx]

bool_val, res_df = find_similiar_sequence(dst_doc=dst_doc, source_doc=source_doc, threshold=0.98)
res_df

,ratio,match,extracted
0,0.980736,"msterdam . teiraf 27jährig . Herr , ev . , aus...","teiraf 27jährig . Herr , ev . , aus erster rhe..."
1,0.982487,"sterdam . teiraf 27jährig . Herr , ev . , aus ...","teiraf 27jährig . Herr , ev . , aus erster rhe..."
2,0.984238,"terdam . teiraf 27jährig . Herr , ev . , aus e...","teiraf 27jährig . Herr , ev . , aus erster rhe..."
3,0.985989,"erdam . teiraf 27jährig . Herr , ev . , aus er...","teiraf 27jährig . Herr , ev . , aus erster rhe..."
4,0.987741,"rdam . teiraf 27jährig . Herr , ev . , aus ers...","teiraf 27jährig . Herr , ev . , aus erster rhe..."
5,0.989492,"dam . teiraf 27jährig . Herr , ev . , aus erst...","teiraf 27jährig . Herr , ev . , aus erster rhe..."
6,0.991243,"am . teiraf 27jährig . Herr , ev . , aus erste...","teiraf 27jährig . Herr , ev . , aus erster rhe..."
7,0.992995,"m . teiraf 27jährig . Herr , ev . , aus erster...","teiraf 27jährig . Herr , ev . , aus erster rhe..."
8,0.994746,". teiraf 27jährig . Herr , ev . , aus erster ...","teiraf 27jährig . Herr , ev . , aus erster rhe..."
9,0.996497,". teiraf 27jährig . Herr , ev . , aus erster r...","teiraf 27jährig . Herr , ev . , aus erster rhe..."


In [19]:

def find_similiar_sequence(dst_doc:str, source_doc:str, threshold:float, return_msk:bool) -> bool:
    """Checks if the extracted content is with a certain threshold in the given document. If a 1:1 match is found it directly returns 
    True - if not levenstein distance is used to check if the extracted string is in the document with a certain threshold of stringsimiliarties
    Parameters:
        dst_doc: str - extracted content
        source_doc: str -  original document
        threshold: float - threshold for stringsimiliarities 
        return_msk: bool - parameter True - just return the mask if val is in doc or false the df with the matches and the ratio will be returned
    Returns:
        True if String is in the original document False if it is not in the document within the threshold   
     """
    
    if dst_doc in source_doc: return True
    
    n = len(dst_doc)
    n_src = len(source_doc)
    i = 0

    best_match_list = []
    best_ratio_list = []
    while n < n_src:
        stringsim = fuzz.ratio(dst_doc, source_doc[i:n])/100
        if stringsim > threshold: 
            best_match_list.append(source_doc[i:n])
            best_ratio_list.append(stringsim)
        i += 1
        n += 1

    matches_df = pd.DataFrame(data={"ratio":best_ratio_list, "match":best_match_list, "extracted":dst_doc})
    if len(matches_df) > 0 and not return_msk:
        return True, matches_df
    elif len(matches_df) > 0:
        return True
    else: 
        return False
    



In [22]:
import pandas as pd
from rapidfuzz import fuzz

class ExtractionValidator:
    def __init__(self,dst_doc:list, source_doc:list, threshold:float ):
        self.dst_doc = dst_doc
        self.source_doc = source_doc
        self.threshold = threshold

    def find_similiar_sequence(self, dst_doc:str, source_doc:str, threshold:float) -> pd.DataFrame:
        """Checks if the extracted content is with a certain threshold in the given document. If a 1:1 match is found it directly returns 
        True - if not levenstein distance is used to check if the extracted string is in the document with a certain threshold of stringsimiliarties
        Parameters:
            dst_doc: str - extracted content
            source_doc: str -  original document
            threshold: float - threshold for stringsimiliarities 
            return_msk: bool - parameter True - just return the mask if val is in doc or false the df with the matches and the ratio will be returned
        Returns:
            pd.DataFrame containing the potential matches
        """
        
        n = len(dst_doc)
        n_src = len(source_doc)
        i = 0

        best_match_list = []
        best_ratio_list = []
        while n < n_src:
            stringsim = fuzz.ratio(dst_doc, source_doc[i:n])/100
            if stringsim > threshold: 
                best_match_list.append(source_doc[i:n])
                best_ratio_list.append(stringsim)
            i += 1
            n += 1

        matches_df = pd.DataFrame(data={"ratio":best_ratio_list, "match":best_match_list, "extracted":dst_doc})
        
        return matches_df
    
    def is_match(self, dst_doc:str, source_doc:str, threshold:float) -> bool:
        """Checks if a match was found. Check if the text is either one to one in the match or it is within the threshold similiar
        Parameters:
            dst_doc: str - extracted content
            source_doc: str -  original document
            threshold: float - threshold for stringsimiliarities 
            return_msk: bool - parameter True - just return the mask if val is in doc or false the df with the matches and the ratio will be returned
        Returns:
            True if String is in the original document False if it is not in the document within the threshold   
        
        
        """
        if dst_doc in source_doc: return True
        match_df = self.find_similiar_sequence(dst_doc, source_doc, threshold)
        if len(match_df) > 0:
            return True
        else:
            return False
        
    def apply_is_match_on_data(self) -> list[bool]:
        """Applys is_match over all data - Can be used as a mask to filter the values that are not correctly extracted
        
        Parameter:
        None

        Return: 
        List of bool values that determining if the given extracted text was is in (True) the source text within the given threshold or not (False)

        """
        found_match_list:list[bool] = [self.is_match(d, s, self.threshold) for d, s in zip(self.dst_doc, self.source_doc)]
        return found_match_list
    
    def calculate_extraction_accuracy(self) -> tuple:
        """Return tuple with the accuracy of the text extraction"""
        n = len(self.dst_doc)
        match_bool = self.apply_is_match_on_data()
        true_positive = sum(match_bool)
        
        return ("Accuracy", true_positive/n)

    
        

    

# Ausgabe der extraktion in Dataframe-Format 

In [28]:

print(dat_joined.text.tolist()[0])
print(dat_joined.plainpagefulltext.tolist()[0])

ExtractionValidator(None, None, 0.98).find_similiar_sequence(dst_doc=dat_joined.text.tolist()[0], source_doc=dat_joined.plainpagefulltext.tolist()[0], threshold=0.8)

Suche für meine Tochter , 18 Jahre , Stelle als angeh . Arbeiterin in kathol . Hause bei Familienanschl . in Boun oder Umgegend . Augebote u . A . K . A . 4368 .
1 — 92 . R Geicch * 1 RRAR ädchen Schuhmachergeselle # D # eres gesucht . Gehr i vontins grsagr Un . si Sautuus b - futge . Mondorf , Sieglor , Markt 15 . Bäckerlehrling . Suche für meinen 16jähr Sehn bei einem katholischen Meister Stelle als Bäcker lohrling , am liebsten in einer vereinigten Bäckerei und Kon # # torel . Angeb . u . M 357 . Junge von 14 — 16 Jahren vom Leude , der etwas pflügen konn , aufs Land gesucht . # Augeb . unt . G . Sp . 4370 . 120 Lehrlinge # # # d Ostern 1918 in Hand werkslehre unterzubringen . Meldungen mit Nachweis der Befühigung zur Ausbildung von Lehrlingen zu richten an des Städtische Waisen haus , Cöln = Sülz , Stl ; gürtel 41 . relle bämsl . Arbeit gesucht . Bonn , Kurfürstenstr . 62 . Zuverl . , kath . , durchaus ekfohrenes Fräulein als geiaßt . Stütze Frou G . Faßbender , heim b . Cöln , Lin

,ratio,match,extracted
0,0.801242,te unter J . F . 4367 . Modes . Suche für mein...,"Suche für meine Tochter , 18 Jahre , Stelle al..."
1,0.807453,e unter J . F . 4367 . Modes . Suche für meine...,"Suche für meine Tochter , 18 Jahre , Stelle al..."
2,0.813665,unter J . F . 4367 . Modes . Suche für meine ...,"Suche für meine Tochter , 18 Jahre , Stelle al..."
3,0.819876,unter J . F . 4367 . Modes . Suche für meine T...,"Suche für meine Tochter , 18 Jahre , Stelle al..."
4,0.826087,nter J . F . 4367 . Modes . Suche für meine To...,"Suche für meine Tochter , 18 Jahre , Stelle al..."
...,...,...,...
60,0.826087,"Jahre , Stelle als angeh . Arbeiterin in kath...","Suche für meine Tochter , 18 Jahre , Stelle al..."
61,0.819876,"Jahre , Stelle als angeh . Arbeiterin in katho...","Suche für meine Tochter , 18 Jahre , Stelle al..."
62,0.813665,"ahre , Stelle als angeh . Arbeiterin in kathol...","Suche für meine Tochter , 18 Jahre , Stelle al..."
63,0.807453,"hre , Stelle als angeh . Arbeiterin in kathol ...","Suche für meine Tochter , 18 Jahre , Stelle al..."


In [23]:
dat_joined = pd.read_pickle("dat_joined.pkl")

validation = ExtractionValidator(dst_doc=dat_joined.text.tolist(), source_doc=dat_joined.plainpagefulltext.tolist(), threshold=0.98)
result = validation.calculate_extraction_accuracy()
print(result)

('Accuracy', 0.9782608695652174)


In [24]:
len(dat_joined)

46

In [30]:
0.022*46


1.012

In [ ]:

still_not_Correct = dat_joined[msk == False]
still_not_Correct


,page_id,text,time,plainpagefulltext
135,2DW55DZZGQDFIT6CBIIFTKK73ARV7ITF-ALTO9398850_D...,"Suchen f . uns . Tochter , Is raelitin , 24 J ...",2025-06-08_20_13,Gschäffter - Verenl für Bergbau und dussstahlf...
149,2EZVCCSPNJZR5754MLRQDYIXY65QRKO5-ALTO7690695_D...,Herrscheftlüsch eingerrichtetes Haus in Barmen...,2025-06-08_20_13,Bekanntmachung . elsregister ist am in das H #...
199,2IIYBCYT6B2SO57MWJ6HQRQZCRAEWU3L-ALTO9548825_D...,"Mitte der Her . in sehr gus Verhältnissen , su...",2025-06-08_20_13,das Handelsregister ist am 14 . September 1920...
251,2LB436MW4E74OIMHD2DTFM4ISNWPPBPF-ALTO1934287_D...,"Landwirt , kath . , Junggeselle , 36 Jahre , B...",2025-06-08_20_13,"Sonntag , den 16 . September 1917 . Statt beso..."
287,2QINTW3X2TQUT2N7KW6QPCM4INRLQ5D4-ALTO9467138_D...,"Eine Gzroßbach , kath , 31 . , Istahlbandarmie...",2025-06-08_20_13,"Der Geutsche Suden . Mitdestes , fast nebelfre..."
395,373GCHWZ25DT4GNR5J44JCNUHG3GVKQX-ALTO8385016_D...,"Glückliche Heirat ! ger Herr , evgl. , großé F...",2025-06-08_20_13,An die Stammaktionäre der Howaldtswerke . Auf ...


In [ ]:
find_similiar_sequence(still_not_Correct.text[135], still_not_Correct.plainpagefulltext[135], threshold=0.75, return_msk=False)

False

In [ ]:
6/425

0.01411764705882353

In [4]:
import pandas as pd
from scripts.information_extractor_lib import ExtractionValidator

dat_joined = pd.read_pickle("dat_joined.pkl")
dat_joined = dat_joined.head(10)
#validation = ExtractionValidator(dst_doc=dat_joined.text.tolist(), source_doc=dat_joined.plainpagefulltext.tolist(), threshold=0.98)
#result = validation.calculate_extraction_accuracy()
#print(result)

validator = ExtractionValidator(dst_doc=None, source_doc=None, threshold=None)
result = validator.find_similiar_sequence(dst_doc=dat_joined.text[0], source_doc=dat_joined.plainpagefulltext[0], threshold=0.98)
result



,ratio,match,extracted
0,0.981366,". Suche für meine Tochter , 18 Jahre , Stelle...","Suche für meine Tochter , 18 Jahre , Stelle al..."
1,0.987578,". Suche für meine Tochter , 18 Jahre , Stelle ...","Suche für meine Tochter , 18 Jahre , Stelle al..."
2,0.993789,"Suche für meine Tochter , 18 Jahre , Stelle a...","Suche für meine Tochter , 18 Jahre , Stelle al..."
3,1.000000,"Suche für meine Tochter , 18 Jahre , Stelle al...","Suche für meine Tochter , 18 Jahre , Stelle al..."
4,0.993789,"uche für meine Tochter , 18 Jahre , Stelle als...","Suche für meine Tochter , 18 Jahre , Stelle al..."
5,0.987578,"che für meine Tochter , 18 Jahre , Stelle als ...","Suche für meine Tochter , 18 Jahre , Stelle al..."
6,0.981366,"he für meine Tochter , 18 Jahre , Stelle als a...","Suche für meine Tochter , 18 Jahre , Stelle al..."


In [31]:
from scripts.information_extractor_lib import InformationExtractor
from scripts.config_file import configs, generation_config
import pandas as pd
from full_workflow_config import fw_config

dat_joined_complete = pd.read_pickle("dat_joined.pkl")

dat_joined = dat_joined_complete.head(5)

ocr_corrector = InformationExtractor(df=dat_joined, page_id_colname="page_id", text_colname="text", output_filename="ocr_corrected_df.csv")
#Set the configs for the task to perform
ocr_corrector.prompt = fw_config["CORRECTION_PROMPT"]
ocr_corrector.model_name = fw_config["model_name"]
data_corrected = ocr_corrector.extract_data_loop(5)


word_completion = InformationExtractor(df=data_corrected, page_id_colname="page_id", text_colname="text", output_filename="ocr_corrected_df.csv")

word_completion.prompt = fw_config["COMPLETION_PROMPT"]
word_completion.model_name = fw_config["model_name"]
data_word_completed = word_completion.extract_data_loop(5)


data_word_completed.columns = ['page_id_completed', 'text_completed', 'time_completed']
merged_df = pd.concat([data_corrected, data_word_completed], axis = 1)
merged_df = merged_df.filter(["text", "text_completed"])
print(merged_df)
merged_df.to_csv("merged_df_100.csv")


                                                text  \
0  Suche für meine Tochter, 18 Jahre, Stelle als ...   
1  Heirats-Gesuche Bei mon aie Gerdprenderch nung...   
2  Junger Mann, 22 Jahre, kath., sucht aufrichtig...   
3  Zwei junge Herren, 1921 Jahre, suchen die Beka...   
4  Teilnahme nach Brückner. Gewissenhafter deutsc...   

                                      text_completed  
0  Suche für meine Tochter, 18 Jahre, Stelle als ...  
1  Heirats-Gesuche Bei mon aie Gerdprenderch Rech...  
2  Junger Mann, 22 Jahre, katholisch, sucht aufri...  
3  Zwei junge Herren, 1921 Jahre, suchen die Beka...  
4  Teilnahme nach Brückner. Gewissenhafter deutsc...  


In [33]:
merged_df

,text,text_completed
0,"Suche für meine Tochter, 18 Jahre, Stelle als ...","Suche für meine Tochter, 18 Jahre, Stelle als ..."
1,Heirats-Gesuche Bei mon aie Gerdprenderch nung...,Heirats-Gesuche Bei mon aie Gerdprenderch Rech...
2,"Junger Mann, 22 Jahre, kath., sucht aufrichtig...","Junger Mann, 22 Jahre, katholisch, sucht aufri..."
3,"Zwei junge Herren, 1921 Jahre, suchen die Beka...","Zwei junge Herren, 1921 Jahre, suchen die Beka..."
4,Teilnahme nach Brückner. Gewissenhafter deutsc...,Teilnahme nach Brückner. Gewissenhafter deutsc...


# Suchen personen für sich selbst oder für Familien angehörige - als annotatiosfrage

In [39]:
ExtractionValidator(None, None, 0.98).find_similiar_sequence(dst_doc=dat_joined.text[1], source_doc=dat_joined.plainpagefulltext[1], threshold=0.98)

,ratio,match,extracted
0,0.981667,. 4340 . “ Heirats - Gesuche Bei mon aie Gerdp...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...
1,0.983333,4340 . “ Heirats - Gesuche Bei mon aie Gerdpr...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...
2,0.985000,4340 . “ Heirats - Gesuche Bei mon aie Gerdpre...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...
3,0.986667,340 . “ Heirats - Gesuche Bei mon aie Gerdpren...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...
4,0.988333,40 . “ Heirats - Gesuche Bei mon aie Gerdprend...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...
5,0.990000,0 . “ Heirats - Gesuche Bei mon aie Gerdprende...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...
6,0.991667,. “ Heirats - Gesuche Bei mon aie Gerdprender...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...
7,0.993333,. “ Heirats - Gesuche Bei mon aie Gerdprenderc...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...
8,0.995000,“ Heirats - Gesuche Bei mon aie Gerdprenderch...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...
9,0.996667,“ Heirats - Gesuche Bei mon aie Gerdprenderch ...,Heirats - Gesuche Bei mon aie Gerdprenderch nu...


# Wie verhindern garbage in garbage out ?  Dropout für nicht kontaktanzeigen oder zu weit extrahierte Textfenster